In [1]:
import json
from pprint import pprint
import matplotlib as mpl
import matplotlib.pyplot as pltfrom 

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np
import pandas as pd

from keras.layers import Flatten
from keras import regularizers

Using TensorFlow backend.


In [21]:
with open('Microblog_Trainingdata.json') as data_file:    
    Microblog_Trainingdata = json.load(data_file)

for i in range(0,len(Microblog_Trainingdata)):
    Microblog_Trainingdata[i]['spans']=Microblog_Trainingdata[i]['spans'][0]
    Microblog_Trainingdata[i]['sentiment score']=float(Microblog_Trainingdata[i]['sentiment score'])
#pprint(Microblog_Trainingdata)
with open('Microblog_Trialdata.json') as data_file:   
    Microblog_Trialdata = json.load(data_file)
    

#pprint(data)
for i in range(0,len(Microblog_Trialdata)):
    Microblog_Trialdata[i]['spans']=Microblog_Trialdata[i]['spans'][0]
    Microblog_Trialdata[i]['sentiment score']=float(Microblog_Trialdata[i]['sentiment score'])

In [22]:
with open('Microblogs_Testdata.json') as data_file:   
    Microblogs_Testdata = json.load(data_file)

In [23]:
for i in Microblog_Trialdata:
    Microblog_Trainingdata.append(i)

In [25]:
def lower_keys(x):
    if isinstance(x, list):
        return [lower_keys(v) for v in x]
    elif isinstance(x, dict):
        return dict((k.lower(), v.lower() if isinstance(v, str) else v) for k, v in x.items())
    else:
        return x
Microblog_Trainingdata=lower_keys(Microblog_Trainingdata)
Microblogs_Testdata=lower_keys(Microblogs_Testdata)

In [26]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [27]:
import string
def stringClean( inputString ) :
    return inputString.translate(str.maketrans('','',string.punctuation)).encode("ascii", errors="ignore").decode()

In [28]:
def removeStopWords(inputString):
    splitString = inputString.split()
    for word in stop:
        for s in splitString:
            if s == word:
                splitString.remove(s)
    return " ".join(splitString)

In [29]:
maxlen = 0

X_train = []
X_trial=[]
Y_trial=[]
X_test = [(removeStopWords(stringClean(str(Microblog['spans'])))) for Microblog in Microblogs_Testdata]
Y_train = []
Y_test = []
output=[]
for Microblog in Microblog_Trainingdata :
    X_train.append(removeStopWords(stringClean(Microblog['spans'])))
    Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
    output.append(Microblog['sentiment score'])
    if maxlen < len(Microblog['spans'].split()) :
        maxlen = len(Microblog['spans'].split())

#for Microblog in Microblog_Trialdata :
 #   X_train.append(Microblog['spans'])
  #  Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
   # if maxlen < len(Microblog['spans'].split()) :
    #    maxlen = len(Microblog['spans'].split())


print(len(X_train)+len(X_test), len(Y_train))

2510 1710


In [31]:
tokenizer = Tokenizer(num_words=maxlen*20)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
data = pad_sequences(sequences, maxlen)
data.shape

(1710, 25)

In [32]:
model = Sequential()
model.add(Embedding(maxlen*20, 32, input_length=maxlen))
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='leakyrelu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.15))
#, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)
model.add(Dense(4, activation='tanh'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 32)            16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 24,457
Trainable params: 24,457
Non-trainable params: 0
_________________________________________________________________


In [33]:
history = model.fit(data, np.array(Y_train), validation_split=0.2, epochs=15,verbose = 1)

Train on 1368 samples, validate on 342 samples
Epoch 1/15
1368/1368 [==============================] - 2s - loss: 1.0505 - acc: 0.4700 - val_loss: 0.6733 - val_acc: 0.6374
Epoch 2/15
1368/1368 [==============================] - 1s - loss: 0.6593 - acc: 0.6265 - val_loss: 0.6524 - val_acc: 0.6374
Epoch 3/15
1368/1368 [==============================] - 1s - loss: 0.6420 - acc: 0.6455 - val_loss: 0.6359 - val_acc: 0.6433
Epoch 4/15
1368/1368 [==============================] - 0s - loss: 0.6236 - acc: 0.6762 - val_loss: 0.6188 - val_acc: 0.7427
Epoch 5/15
1368/1368 [==============================] - 0s - loss: 0.5793 - acc: 0.7281 - val_loss: 0.5648 - val_acc: 0.7544
Epoch 6/15
1368/1368 [==============================] - 0s - loss: 0.5084 - acc: 0.7931 - val_loss: 0.4945 - val_acc: 0.7778
Epoch 7/15
1368/1368 [==============================] - 1s - loss: 0.4472 - acc: 0.8224 - val_loss: 0.4873 - val_acc: 0.7836
Epoch 8/15
1368/1368 [==============================] - 1s - loss: 0.4055 - ac

In [35]:
sequences = tokenizer.texts_to_sequences(np.array(X_test))
data = pad_sequences(sequences, maxlen)
predictions = model.predict(data)
for i in range(len(predictions)):
    print(Microblogs_Testdata[i]['spans'],"%.3f" % predictions[i])

consumers keep cautious stance -0.140
close $ctrp @46.16 from 43.55 entry +6%;$;+6% -0.034
every reason to be bullish 0.975
$panw need anoth $1 to all time high -0.478
long setup 0.976
will be a solid long term investmen;$;#passive;$;also added $sbux today;$; #income 0.979
recall 0.569
marriott wins new starwood deal by raising cash portion of bid 0.977
time to embrace dividend growth 0.959
bullshit! all #applestores are empty in #china;$;$ms must be preparing to liquidate $aapl 0.171
5-star analyst robert derrington from wunderlich reiterated a buy 0.967
excited;$;big +value;$;hoping 0.607
oil to break out;$;adding chevron 0.946
unusual call activity 0.980
china's tourist bonanza isn't going anywhere 0.402
0.15 0.607
not guaranteed 0.607
potential continuation uptrend 0.973
longed 0.607
absolute garbage still up;$;stores totally empty;$;stock mispriced 0.607
buy the dip' 0.858
weakest in todays dt watchlist 0.953
we see more reasons to be cautious -0.456
breaking out 0.895
$bbry smack

trails $msft $orcl $goog $crm $adbe 0.964
$cytr af bashing 0.607
out of $amgn one year later;$;i'm tired of waiting here 0.016
was right to reject the offer -0.321
bot more $ung puts 0.597
right at the top of thre cloud -0.060
buying $tza calls 0.956
closed friday with the rs line in new highs 0.980
rising steel demand and higher prices in both steel and coal;$;cash magnet 0.960
pie in the sky 0.975
fed cuts rates forecast;$;our calls go 100% 0.974
adding to the puts 0.521
anyone who stayed long $aapl made plenty $ 0.953
like this one a lot. -0.079
unusual call activity 0.980
actually lost .03c per share 0.607
our updated #core4 portfolio 0.961
dips being bought up real quick 0.939
highest call to put ratio 0.975
downside breakout looks to be coming soon 0.980
equities recently tagged #undervalued 0.880
maintains neutral rating at jp morgan;$;pt cut 0.549
heapest #stock by pe in television services industry;$;pe 8.18 vs.avg. 26.864779 0.784
hell yeah ill take that fill all day long bab

In [36]:
df=pd.DataFrame.from_dict(Microblogs_Testdata)

In [37]:
df['Predicted Sentiment Score']=predictions
df.head()

,cashtag,id,source,spans,Predicted Sentiment Score
0,$cost,709723193125175300,twitter,consumers keep cautious stance,-0.139868
1,$ctrp,719529094652698600,twitter,close $ctrp @46.16 from 43.55 entry +6%;$;+6%,-0.033936
2,$intc,39048670,stocktwits,every reason to be bullish,0.975283
3,$panw,37048093,stocktwits,$panw need anoth $1 to all time high,-0.478138
4,$jrcc,6207860,stocktwits,long setup,0.976049


In [39]:
df.to_csv('TestData.csv',index=False)